In [1]:
import json
import os
from datetime import datetime

import src.data.Dataset as dt

In [ ]:
required_paths = ["/ai4eutils", "/CameraTraps", "/yolov5"]
python_path = os.environ.get("PYTHONPATH", "")
root_path = os.getcwd()

for path in required_paths:
    if not any(p.endswith(path) for p in python_path.split(":")):
        python_path += f":{root_path}/data/external{path}"

os.environ["PYTHONPATH"] = python_path

!echo "PYTHONPATH: $PYTHONPATH"

In [ ]:
threshold = float(0.01)

OUTPUT_PATH = os.path.abspath("./resources/json")
MODEL_PATH = os.path.abspath("./models/MegaDetector/md_v5a.0.0.pt")
# MODEL_PATH = os.path.abspath("./models/MegaDetector/md_v5b.0.0.pt")
# MODEL_PATH = os.path.abspath("./models/MegaDetector/md_v4.1.0.pb")

DATASET_PATH = os.path.abspath("./dataset/emptyNonEmptyDatasetCropped")
CSV_PATH = os.path.abspath("./data/interim/20Images_binary.csv")
dataset = dt.load_from_csv(CSV_PATH)

dataset['file_name_abspath'] = dataset['file_name'].apply(lambda x: os.path.join(DATASET_PATH, x))
IMAGES_PATH_JSON = os.path.join(OUTPUT_PATH, "dataset_file_paths.json")

os.makedirs(OUTPUT_PATH, exist_ok=True)

with open(IMAGES_PATH_JSON, "w") as f:
    json.dump(dataset['file_name_abspath'].tolist(), f, indent=1)


OUTPUT_FILE_PATH = os.path.join(OUTPUT_PATH, "detections.json")

print()
print(f"MODEL_PATH:       {MODEL_PATH}")
print(f"DATASET_PATH:     {DATASET_PATH}")
print(f"IMAGES_PATH_JSON: {IMAGES_PATH_JSON}")
print(f"OUTPUT_PATH:      {OUTPUT_PATH}")
print(f"OUTPUT_FILE_PATH: {OUTPUT_FILE_PATH}")

In [4]:
detection_inicialitation_time = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")

In [ ]:
command = f'python data/external/CameraTraps/detection/run_detector_batch.py "{MODEL_PATH}" "{IMAGES_PATH_JSON}" "{OUTPUT_FILE_PATH}" --recursive --threshold "{threshold}"'
os.system(command)

In [ ]:
with open(OUTPUT_FILE_PATH, "r") as file:
    data = json.load(file)

for image in data["images"]:
    image["file"] = (
        image["file"]
        .replace(
            DATASET_PATH + "/",
            "",
        )
    )

info = {
    "detection_initial_time": detection_inicialitation_time,
    "detection_completion_time": data["info"]["detection_completion_time"],
    "format_version": data["info"]["format_version"],
    "detector": data["info"]["detector"],
    "detector_threshold": threshold,
    "detector_metadata": data["info"]["detector_metadata"],
}

final_output = {
    "images": data["images"],
    "detection_categories": data["detection_categories"],
    "info": info,
}

threshold_str = str(threshold).replace(".", "_")
json_name = f"{len(data['images'])}_images_{threshold_str}_threshold.json"
model_name = os.path.basename(MODEL_PATH).split(".")[0]
tmp_path = os.path.join(OUTPUT_PATH, model_name)
os.makedirs(tmp_path, exist_ok=True)
NEW_OUTPUT_FILE_PATH = os.path.join(tmp_path, json_name)


with open(NEW_OUTPUT_FILE_PATH, "w") as f:
    json.dump(final_output, f, indent=1)
print(f"Output file saved at {NEW_OUTPUT_FILE_PATH}")

os.remove(OUTPUT_FILE_PATH)
print(f"Output file deleted: {OUTPUT_FILE_PATH}")

os.remove(IMAGES_PATH_JSON)
print(f"Images paths json file deleted: {IMAGES_PATH_JSON}")
